## Notebook 1
# Extraindo coordenadas de decretos e leis

Autores:
* Pedro C. de Siracusa
* André G. Coutinho
* Waira Saravia Machida
* Augusto Arcela
* Yan Felipe Soares

---

Neste estudo exploramos decretos e leis que embasam a criação de UCs, em âmbitos Federal e Municipal.
O objetivo é extrair as coordenadas que delimitam as UCs, de forma que possamos posteriormente projetar o polígono em um ambiente SIG. 
Os **principais desafios**:

* Os documentos estão no formato `.pdf`, não muito amigável para a extração de dados. Algumas bibliotecas auxiliam na extração de texto de arquivos `.pdf`, mas elas nem sempre funcionam como esperamos. Isso porque estes arquivos podem ter sido codificados de muitas formas diferentes, e nem sempre conseguimos fazer a extração.

* Os documentos não seguem um padrão estrutural, o que significa que nem sempre a informação que desejamos está no mesmo lugar ou se apresenta da mesma forma. Isso dificulta a extração inteiramente automatizada da informação nestes documentos.

* A delimitação das UCs é apresentada de forma descritiva nestes documentos. Em alguns poucos casos as coordenadas são organizadas em uma tabela, facilitando um pouco a extração.

Vamos explorar a **Lei No.4771** da Casa Civil da Presidencia da Republica, que determina os limites do Parque Nacional da Serra da Canastra.
O documento está nomeado com o`l4771.pdf`, localizado dentro do diretório `data`, na raiz do projeto.

O primeiro passo foi extrair (manualmente) a parte do documento que descreve a delimitação da UC.
Armazenamos esta informação no arquivo `l4771_limits.txt`, para facilitar a leitura e extração de coordenadas do texto.

In [225]:
fpath = '../data/decretos/limits/l4771_limits.txt'

Agora vamos abrir o arquivo, ler as linhas de texto e armazenar numa variável `lines`. Na célula abaixo usamos uma estrutura do *Python* que se chama [context manager](https://docs.python.org/3/reference/compound_stmts.html#with), para facilitar a abertura/fechamento de arquivos.

In [226]:
with open(fpath, encoding='UTF-8') as f:
    lines =''.join( f.readlines() )

Vejamos os 1000 primeiros caracteres do texto:

In [227]:
lines[:1000]+' (...)'

'Art. 2º O Parque Nacional da Serra da Canastra, com uma área estimada em 200.000 ha (duzentos mil hectares), é\ndelimitado por ma linha assim definida: no extremo oeste, inicia-se no Ribeirão do Engano, 2,5km acima de sua foz, na\nrepresa de Peixotos, na altura do meridiano de longitude 47º 00\' 00" W e do paralelo de latitude 20º 11\' 30" S (Ponto 1);\nseguindo por esse Ribeirão acima até suas cabeceiras, junto ao ponto de latitude 20º 05\' 20" S e longitude 46º 55\' 10" W\n(Ponto 2); segue em linha reta, rumo ao norte pelo meridiano 46º 55\' 10" W, numa distância de 7km, até atingir o ponto\nde latitude 20º 04\' 32" S (Ponto 3); desse ponto, vira à direita e segue em linha reta numa extensão de 18,5km, até\natingir o ponto de latitude 20º 06\' 30" S e longitude 46º 45\' 40" W, na altura da Fazenda da Cachoeira (Ponto 4); daí, vira\nà direita, no sentido sudeste, seguindo numa linha reta com extensão de 11km até encontrar a interseção da latitude 20º 08\' 30" S e \ncom a longitude 46

Neste caso, as coordenadas parecem estar descritas em UTM seguindo o padrão `E-X.XXX,XXX e N-X.XXX,XXX`.
Para extrair o texto que segue este padrão vamos usar **expressões regulares**.

In [228]:
len(lines)

5702

---

## Regular Expressions

Expressões regulares servem para definirmos padrões de busca em um texto. Uma vez definido o padrão que desejamos buscar, o texto é percorrido tentando encontrar "casamentos", que podem então ser retornados. Usaremos expressões regulares para tentar extrair as coordenadas, assumindo que elas tenham sido inseridas em um padrão relativamente estável.

Alguns exemplos e guias de como se utilizar expressões regulares: 
1. https://medium.com/tech-tajawal/regular-expressions-the-last-guide-6800283ac034
1. https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285
1. http://www.rexegg.com/regex-quickstart.html#ref

O *Python* já fornece um módulo que nos permite trabalhar com expressões regulares de forma facilitada. O nome do módulo é `re`, e para usá-lo precisamos importá-lo. Veja [aqui](https://docs.python.org/3/howto/regex.html) como usar expressões regulares com o módulo `re`.

In [6]:
import re

#### Primeira tentativa

Abaixo vamos fazer uma primeira tentativa de expressão para capturar as coordenadas. Guardamos a expressão em uma variável de nome `expr`.

In [229]:
expr = '[0-9]{1,2}º [0-9]{1,2}\'? ?[0-9]{0,2}"? [WS]'

Agora vamos usar a função `findall` para encontrar todos os *"matches"* para esta expressão. As coordenadas obtidas são armazenadas na variável `coords`. Para melhorar a visualização vou olhar apenas para as 10 primeiras coordenadas que foram obtidas.

In [230]:
coords = re.findall( expr , lines )

In [231]:
coords

['47º 00\' 00" W',
 '20º 11\' 30" S',
 '20º 05\' 20" S',
 '46º 55\' 10" W',
 '46º 55\' 10" W',
 '20º 04\' 32" S',
 '20º 06\' 30" S',
 '46º 45\' 40" W',
 '20º 08\' 30" S',
 '46º 39\' 55" W',
 '20º 08\' 30" S',
 '46º 35\' 15" W',
 '20º 08\' 20" S',
 '46º 28\' 32" W',
 '46º 23\' 44" W',
 '20º 12\' 00" S',
 '46º 20\' 52" W',
 '20º 18\' 00" S',
 '46º 25\' 27" W',
 '20º 19\' 43" S',
 '46º 25\' 51" W',
 '20º 16\' 55" S',
 "46º 03' W",
 '20º 18\' 37" S',
 '46º 31\' 03" W',
 '20º 20\' 27" S',
 '46º 17\' 09" W',
 '20º 31\' 32" S',
 '46º 15\' 00" W',
 '20º 30\' 27" S',
 '46º 12\' 02" W',
 '20º 32\' 09" S',
 '20º 32\' 09" S',
 '20º 35\' 29" S',
 '46º 13\' 18" W',
 '20º 38\' 55" S',
 '46º 18\' 51" W',
 '46º 30\' 02" W',
 '20º 30\' 22" S',
 '20º 37\' 35" S',
 '46º 30\' 03" W',
 '46º 33\' 21" W',
 '20º 30\' 29" S',
 '20º 16\' 48" S',
 '46º 52\' 17" W',
 '46º 43\' 14" W',
 '20º 18\'55" S',
 '46º 43\' 14" W',
 '20º 17\' 08" S',
 '46º 57\' 25" W',
 '20º 11\' 30" S',
 '20º 11\' 30" S']

Quantas coordenadas foram obtidas ao todo?

In [232]:
len(coords)

52

Um problema deste método que usamos acima é que fica um pouco difícil validar se as coordenadas realmente estão sendo extraídas em pares. Caso uma delas não tenha sido capturada, isso pode gerar uma deformação do polígono extraído.

#### Segunda tentativa

Vamos fazer uma segunda tentativa separando os pares em latitude e longitude. Agora, queremos incluir também as palavras 'longitude' (ou 'meridiano', em uma exceção) e 'latitude', que parece sempre indicar um novo par de coordenadas no texto original. Dessa forma fica mais fácil de avaliar se as coordenadas estão formando os pares corretamente.

In [262]:
expr_long = 'longitude *[0-9]{1,2}º [0-9]{1,2}\'? ?[0-9]{0,2}"? W|meridiano +[0-9]{1,2}º [0-9]{1,2}\'? ?[0-9]{0,2}"? W'

In [263]:
coords_long = re.findall( expr_long, lines )

In [264]:
coords_long # observando apenas as primeiras 10 entradas

['longitude 47º 00\' 00" W',
 'longitude 46º 55\' 10" W',
 'meridiano 46º 55\' 10" W',
 'longitude 46º 45\' 40" W',
 'longitude 46º 39\' 55" W',
 'longitude 46º 35\' 15" W',
 'longitude 46º 28\' 32" W',
 'longitude 46º 23\' 44" W',
 'longitude 46º 20\' 52" W',
 'longitude 46º 25\' 27" W',
 'longitude 46º 25\' 51" W',
 "longitude 46º 03' W",
 'longitude 46º 31\' 03" W',
 'longitude 46º 17\' 09" W',
 'longitude 46º 15\' 00" W',
 'longitude 46º 12\' 02" W',
 'longitude 46º 13\' 18" W',
 'longitude 46º 18\' 51" W',
 'longitude 46º 30\' 02" W',
 'longitude 46º 30\' 03" W',
 'longitude 46º 33\' 21" W',
 'longitude 46º 52\' 17" W',
 'longitude 46º 43\' 14" W',
 'longitude 46º 43\' 14" W',
 'longitude 46º 57\' 25" W']

Podemos simplificar a parte da lista que nos interessa através desse código omitindo as palavras que as antecedem se determinarmos W (para longitude) ou S (para latitude).

In [266]:
expr_lat = 'latitude d?e? ?[0-9]{1,2}º [0-9]{1,2}\'? ?[0-9]{0,2}"? S'

In [267]:
coords_lat = re.findall( expr_lat, lines )

In [268]:
coords_lat

['latitude 20º 11\' 30" S',
 'latitude 20º 05\' 20" S',
 'latitude 20º 04\' 32" S',
 'latitude 20º 06\' 30" S',
 'latitude 20º 08\' 30" S',
 'latitude 20º 08\' 30" S',
 'latitude 20º 08\' 20" S',
 'latitude de 20º 12\' 00" S',
 'latitude 20º 18\' 00" S',
 'latitude 20º 19\' 43" S',
 'latitude 20º 16\' 55" S',
 'latitude 20º 18\' 37" S',
 'latitude 20º 20\' 27" S',
 'latitude 20º 31\' 32" S',
 'latitude 20º 30\' 27" S',
 'latitude de 20º 32\' 09" S',
 'latitude 20º 32\' 09" S',
 'latitude 20º 35\' 29" S',
 'latitude 20º 38\' 55" S',
 'latitude 20º 30\' 22" S',
 'latitude 20º 37\' 35" S',
 'latitude 20º 30\' 29" S',
 'latitude 20º 16\' 48" S',
 'latitude 20º 18\'55" S',
 'latitude 20º 17\' 08" S',
 'latitude de 20º 11\' 30" S',
 'latitude 20º 11\' 30" S']

In [304]:
expr_long0 = '[0-9]{1,2}º [0-9]{1,2}\'? ?[0-9]{0,2}"? W|[0-9]{1,2}º [0-9]{1,2}\'? ?[0-9]{0,2}"? W'

In [305]:
coords_long0 = re.findall( expr_long0, lines )

In [306]:
coords_long0

['47º 00\' 00" W',
 '46º 55\' 10" W',
 '46º 55\' 10" W',
 '46º 45\' 40" W',
 '46º 39\' 55" W',
 '46º 35\' 15" W',
 '46º 28\' 32" W',
 '46º 23\' 44" W',
 '46º 20\' 52" W',
 '46º 25\' 27" W',
 '46º 25\' 51" W',
 "46º 03' W",
 '46º 31\' 03" W',
 '46º 17\' 09" W',
 '46º 15\' 00" W',
 '46º 12\' 02" W',
 '46º 13\' 18" W',
 '46º 18\' 51" W',
 '46º 30\' 02" W',
 '46º 30\' 03" W',
 '46º 33\' 21" W',
 '46º 52\' 17" W',
 '46º 43\' 14" W',
 '46º 43\' 14" W',
 '46º 57\' 25" W']

No texto original, existem no total 2361 vértices. Quantos vértices conseguimos extrair com o método acima?

In [307]:
len(coords_long0)

25

---

In [270]:
expr_lat0 = '[0-9]{1,2}º [0-9]{1,2}\'? ?[0-9]{0,2}"? S'

In [271]:
coords_lat0 = re.findall( expr_lat0, lines )

In [272]:
coords_lat0

['20º 11\' 30" S',
 '20º 05\' 20" S',
 '20º 04\' 32" S',
 '20º 06\' 30" S',
 '20º 08\' 30" S',
 '20º 08\' 30" S',
 '20º 08\' 20" S',
 '20º 12\' 00" S',
 '20º 18\' 00" S',
 '20º 19\' 43" S',
 '20º 16\' 55" S',
 '20º 18\' 37" S',
 '20º 20\' 27" S',
 '20º 31\' 32" S',
 '20º 30\' 27" S',
 '20º 32\' 09" S',
 '20º 32\' 09" S',
 '20º 35\' 29" S',
 '20º 38\' 55" S',
 '20º 30\' 22" S',
 '20º 37\' 35" S',
 '20º 30\' 29" S',
 '20º 16\' 48" S',
 '20º 18\'55" S',
 '20º 17\' 08" S',
 '20º 11\' 30" S',
 '20º 11\' 30" S']

In [285]:
len(coords_lat0)

25

A diferença entre o # de latitudes e longitudes se deve por que no documento a pessoa se referiu a coordenadas repetidas para passar do ponto 16 para o 17 e do 25 para o 1 (fechando o polígono). Ambos apresentam a partícula "de", mas uma coordenada única também a apresenta (8).

# Remoção de duplicatas

A função definida abaixo remove duplicatas na ordem em que a lista foi criada. Ela cria um objeto seen para a função set() e seen_add para a função set.add() de tal forma que estas funções não precisam ser resolvidas a cada iteração (isso ocorre pela natureza dinâmica de python). De outra forma seria mais custoso em termos de processamento computacional, além de que seen_add poderia mudar entre as iterações, o que não é o que nós queremos.

In [1]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [276]:
coords_lat_norep = f7(coords_lat0)

In [278]:
coords_lat_norep

['20º 11\' 30" S',
 '20º 05\' 20" S',
 '20º 04\' 32" S',
 '20º 06\' 30" S',
 '20º 08\' 30" S',
 '20º 08\' 20" S',
 '20º 12\' 00" S',
 '20º 18\' 00" S',
 '20º 19\' 43" S',
 '20º 16\' 55" S',
 '20º 18\' 37" S',
 '20º 20\' 27" S',
 '20º 31\' 32" S',
 '20º 30\' 27" S',
 '20º 32\' 09" S',
 '20º 35\' 29" S',
 '20º 38\' 55" S',
 '20º 30\' 22" S',
 '20º 37\' 35" S',
 '20º 30\' 29" S',
 '20º 16\' 48" S',
 '20º 18\'55" S',
 '20º 17\' 08" S']

In [277]:
len(coords_lat_norep)

23

Removeu mais do que o necessário, já que os pontos 1 e 25 apresentam a mesma latitude (além da coordenada repetida para fechar o polígono), assim como os 5 e 6. Logo, queremos remover apenas as entradas 16 e 26 (referentes às coordenadas repetidas que orientam os pontos próximos 16 > 17 e 25 > 1 -- lembre-se que a primeira entrada é 0). A função abaixo determina a remoção de itens ordenados da lista:

In [290]:
indices = 16, 26


for i in sorted(indices, reverse=True):
    del coords_lat[i]

In [291]:
coords_lat

['latitude 20º 11\' 30" S',
 'latitude 20º 05\' 20" S',
 'latitude 20º 04\' 32" S',
 'latitude 20º 06\' 30" S',
 'latitude 20º 08\' 30" S',
 'latitude 20º 08\' 30" S',
 'latitude 20º 08\' 20" S',
 'latitude de 20º 12\' 00" S',
 'latitude 20º 18\' 00" S',
 'latitude 20º 19\' 43" S',
 'latitude 20º 16\' 55" S',
 'latitude 20º 18\' 37" S',
 'latitude 20º 20\' 27" S',
 'latitude 20º 31\' 32" S',
 'latitude 20º 30\' 27" S',
 'latitude de 20º 32\' 09" S',
 'latitude 20º 35\' 29" S',
 'latitude 20º 38\' 55" S',
 'latitude 20º 30\' 22" S',
 'latitude 20º 37\' 35" S',
 'latitude 20º 30\' 29" S',
 'latitude 20º 16\' 48" S',
 'latitude 20º 18\'55" S',
 'latitude 20º 17\' 08" S',
 'latitude de 20º 11\' 30" S']

In [296]:
len(coords_long)

25

Chegamos ao número que queríamos, já que são 25 pontos

In [298]:
type(coords_lat)

list

## Estruturando coordenadas

Agora que temos duas listas de strings armazenando as coordenadas na forma `XXº XX' XX" S/W `, precisamos apenas juntas os valores de coordenadas em pares. Por conta disso, vamos representá-las como tuplas. A função zip abaixo realiza esta tarefa, mas como ela cria um objeto 'zip', devemos listá-lo para visualizar o que há gravado:

In [309]:
coords_tuplas = list(zip(coords_lat0, coords_long0))

In [310]:
coords_tuplas

[('20º 11\' 30" S', '47º 00\' 00" W'),
 ('20º 05\' 20" S', '46º 55\' 10" W'),
 ('20º 04\' 32" S', '46º 55\' 10" W'),
 ('20º 06\' 30" S', '46º 45\' 40" W'),
 ('20º 08\' 30" S', '46º 39\' 55" W'),
 ('20º 08\' 30" S', '46º 35\' 15" W'),
 ('20º 08\' 20" S', '46º 28\' 32" W'),
 ('20º 12\' 00" S', '46º 23\' 44" W'),
 ('20º 18\' 00" S', '46º 20\' 52" W'),
 ('20º 19\' 43" S', '46º 25\' 27" W'),
 ('20º 16\' 55" S', '46º 25\' 51" W'),
 ('20º 18\' 37" S', "46º 03' W"),
 ('20º 20\' 27" S', '46º 31\' 03" W'),
 ('20º 31\' 32" S', '46º 17\' 09" W'),
 ('20º 30\' 27" S', '46º 15\' 00" W'),
 ('20º 32\' 09" S', '46º 12\' 02" W'),
 ('20º 35\' 29" S', '46º 13\' 18" W'),
 ('20º 38\' 55" S', '46º 18\' 51" W'),
 ('20º 30\' 22" S', '46º 30\' 02" W'),
 ('20º 37\' 35" S', '46º 30\' 03" W'),
 ('20º 30\' 29" S', '46º 33\' 21" W'),
 ('20º 16\' 48" S', '46º 52\' 17" W'),
 ('20º 18\'55" S', '46º 43\' 14" W'),
 ('20º 17\' 08" S', '46º 43\' 14" W'),
 ('20º 11\' 30" S', '46º 57\' 25" W')]

In [311]:
len(coords_tuplas)

25

Agora precisamos validar e corrigir as coordenadas na lista `coords`.

---

## Validando as coordenadas